In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from dotenv import dotenv_values
from hrflow import Hrflow
import re
from crawler import Crawler

NameError: name 'hhhh' is not defined

In [9]:

PATH = 'C:\chromedriver_win32\chromedriver.exe'

# Get The last version of the chromedriver in this link:
# https://chromedriver.storage.googleapis.com/index.html


settings = {
    **dotenv_values(".env")
}

c = Crawler(settings)

driver = c.get_driver()

# job_0aa9dc37ec9aa503
print("*************************************")
driver.get("https://uk.indeed.com/jobs?l=All")


total_jobs = int(re.sub("\D", "", driver.find_element(
    By.CLASS_NAME, "jobsearch-JobCountAndSortPane-jobCount").text))

count_jobs = 15  # count jobs per Page
# total_pages = 66  # Maximum number of pages
total_pages = int(total_jobs // count_jobs + 1)
total_pages = 2  # number of pages for test

print("Total jobs: %d" % total_jobs)
print("count_jobs: %d" % count_jobs)
print("total_pages: %d" % total_pages)

for page in range(0, total_pages):

    print(f"******************** Page {page + 1} ************************")

    if page != 0:
        driver.get("https://uk.indeed.com/jobs?q=&l=All&start=%s" %
                   (page * count_jobs))

    tags_jobTitles = driver.find_elements(By.CLASS_NAME, "jcs-JobTitle")

    # Get ID and href attributes of <a /> tags
    links = [{"id": a.get_attribute("id"), "url": a.get_attribute(
        "href")} for a in tags_jobTitles]

    for a in links:
        print("=====================================================")
        reference = a["id"]
        url = a["url"]
        print("reference:", reference)
        print("url:", url)

        # Browse to specific job URL to get specific data on the page
        driver.get(url)

        # Get Job Name
        name = driver.find_element(
            By.CLASS_NAME, "jobsearch-JobInfoHeader-title-container").text

        print("name:", name)

        # Get all Tags Names (salary: 30K-40K, Job type: Fixed term contract, etc.)
        tags_job = []
        try:
            tag_list = driver.find_elements(
                By.CSS_SELECTOR, ".css-4m8ia3:not(.jobsearch-JobDescriptionSection-title)")

            if len(tag_list) > 0:
                tags_job = [
                    {"name": tag.find_element(By.CSS_SELECTOR, ".css-fhkva6").text, "value": ', '.join(
                        [v.text for v in tag.find_elements(By.CSS_SELECTOR, ".css-tvvxwd")])} for tag in tag_list]
        except NoSuchElementException:
            tag_list = None

        tags_job.append({"name": "company", "value": driver.find_element(
            By.XPATH, "//*[@data-company-name='true']").text})

        # Get Description
        description = driver.find_element(By.ID, 'jobDescriptionText').text

        # Initial JSON representation of the job
        json_job = c.format_job()

        json_job["agent_key"] = reference
        json_job["reference"] = reference
        json_job["name"] = name
        json_job["url"] = url
        json_job["sections"] = [
            {"name": "description", "title": "Description",
                "description": description}
        ],

        json_job["tags"] = tags_job

        # Verify that the job exists in the database
        is_exists = c.is_job_exists(reference)
        if not is_exists:
            # Save JSON job data into hrflow.ai database (API).
            code = c.save(json_job)
            print("code: ", code)
            print("verify_job:", is_exists)
        else:
            print("verify_job:", is_exists)
            # print("verify_job: ", list(verify_job.items())[:4])


# Check some jobs exists in hrflow.ai database (API).

print(" Check some jobs exists in hrflow.ai database: ")
references = [
    'job_0aa9dc37ec9aa503',
    'job_fd3244632c3893c0',
    'job_ee5cacc50ab8d624'
]

for ref in references:

    print("=====================")
    print(f'{ref} : ', c.get_job(ref))


print(f"\ndriver process is closing...")

c.close()
print(f"\n********************* finished ***********************")


*************************************
Total jobs: 577852
count_jobs: 15
total_pages: 2
******************** Page 1 ************************
reference: job_f7ea229503df7a35
url: https://uk.indeed.com/rc/clk?jk=f7ea229503df7a35&fccid=42902ed7252fb8a2&vjs=3
name: Online Rater for AI Models | Farsi and English | Remote, Part Time, Work from Home
verify_job: True
reference: job_c7304052f01111c8
url: https://uk.indeed.com/rc/clk?jk=c7304052f01111c8&fccid=9474516a9efc22f2&vjs=3
name: Receptionist
verify_job: True
reference: job_0aa9dc37ec9aa503
url: https://uk.indeed.com/rc/clk?jk=0aa9dc37ec9aa503&fccid=5637f376797d3c48&vjs=3
name: Email Marketing Assistant
verify_job: True
reference: job_3db748ddcf374946
url: https://uk.indeed.com/rc/clk?jk=3db748ddcf374946&fccid=45eca27a1f2b4add&vjs=3
name: Europe Luxury Travel Consultant
verify_job: True
reference: job_11c3ea77bd41efa9
url: https://uk.indeed.com/rc/clk?jk=11c3ea77bd41efa9&fccid=42f1e4ca72488e15&vjs=3
name: Amazon Content Specialist
verify_

job_fd3244632c3893c0 :  {'id': 1243072, 'key': 'f2efb8d83120ddb68d727db428df70a17fd0b5d3', 'reference': 'job_fd3244632c3893c0', 'board_key': 'd0ee1d408b3b6c2831876f2e636d6f3c68e2bdce', 'board': {'key': 'd0ee1d408b3b6c2831876f2e636d6f3c68e2bdce', 'name': 'test_gayl_nabil', 'type': 'api', 'subtype': 'python', 'environment': 'production'}, 'name': 'JUNIOR CREATIVE', 'url': 'https://uk.indeed.com/rc/clk?jk=fd3244632c3893c0&fccid=ec9ea70f1e5ed747&vjs=3', 'picture': None, 'summary': '', 'location': {'text': None, 'lat': None, 'lng': None, 'gmaps': None, 'fields': []}, 'archive': None, 'archived_at': None, 'updated_at': '2023-06-17T20:18:14+0000', 'created_at': '2023-06-17T20:18:14+0000', 'sections': [[{'name': 'description', 'title': 'Description', 'description': "WING, a creative production arm of Little Dot Studios, has a new role for an ambitious Junior Creative to bring their killer ideas and passion to our content ideation and output and help us to deliver high-end films and commercials


********************* finished ***********************
